Yay! Exercises!

# Imports

In [18]:
import pandas as pd
#from sklearn.linear_model import LinearRegression
import numpy as np
import hashlib # for grading purposes
%matplotlib inline
from sklearn.impute import KNNImputer

### You're hired

You've been hired as the data scientist for a supermarket chain that wants to start extracting insights from their data. First, you'll start with a specific store.

Let's get our data: `store.csv` under `data/` folder

> Important: save the dataframe into the `store` variable

In [27]:
url = 'https://raw.githubusercontent.com/samsung-ai-course/6-7-edition/refs/heads/main/Data%20Wrangling/Timeseries%20%26%20DoubleIndex/TimeSeries/data/store.csv'

store = pd.read_csv(url)
store.head()

store.iloc[25]

,25
date,06-07-2013
customers,1186


## Exercise 1: Index and datetime

#### 1.1) Make the index the datetime of the dates.

In [28]:
# We expect the solution to be a dataframe

store['date'] = pd.to_datetime(store['date'], format='%d-%m-%Y')
store = store.set_index('date').sort_index()

store.head(5)


,customers
date,
2013-01-02,2111
2013-01-03,1833
2013-01-04,1863
2013-01-05,1509
2013-01-06,520


In [29]:
expected_hash = '660d9054fe3a6cbcfb77e2647932e3c41ff5acab9fc4d162fdc448c7c8e6ccc2'
assert hashlib.sha256(str(store.iloc[28].name).encode()).hexdigest() == expected_hash
assert hashlib.sha256(str(store.index.dtype).encode()).hexdigest() == '261738f2e43a1c47a16f043b46deb993943d61f4a2bbe5ef4b03c3fb1af362b5'


# clue: if this assert is failing, and your iloc[25].name is '2017-01-7',
# then you are missing the "best practices" part.
# What did we say in the Learning notebook about this?

## Exercise 2: Time series preprocessing

#### 2.1) Accounting for missing days

Sometimes datasets don't have rows corresponding to all timestamps, as a data scientist you should know if this is the case. Copy `store` to a new variable called `store_complete`, with no gap days. Fill the missing data with nulls.

In [35]:
# copy store to store_complete
store_complete = store.copy()

# change store_complete index so that it contains each of the days in the time range [2013-01-01, 2017-07-31]
# fill missing data with nulls

# YOUR CODE HERE
interval = slice('2013-01-01', '2017-07-31')
store_complete = store.loc[interval].asfreq('D')
store_complete

,customers
date,
2013-01-02,2111.0
2013-01-03,1833.0
2013-01-04,1863.0
2013-01-05,1509.0
2013-01-06,520.0
...,...
2017-07-27,1729.0
2017-07-28,1848.0
2017-07-29,1251.0


In [38]:
#### check number of nulls
assert store_complete.isnull().sum()[0] != 0, "You have 0 null values in the dataset! Remember that each missing day should correspond to a null in customers."
assert store_complete.isnull().sum()[0] == 11, "You should have found 11 days with missing data, no more no less."

#### check store_complete dataframe integrity
assert store_complete.shape[0] != 1676, "Did you fill the index with the missing days?"
assert store_complete.shape[0] == 1672, "The number of rows is not the expected."
assert store_complete.shape[1] == 1, "You shouldn't change the number of columns."
assert str(store_complete[store_complete.customers.isnull()].index[6])[:10] == '2016-01-01', "Do you have all the missing days? Is the index is ordered?"

<ipython-input-38-f0b280e1b569>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  assert store_complete.isnull().sum()[0] != 0, "You have 0 null values in the dataset! Remember that each missing day should correspond to a null in customers."
<ipython-input-38-f0b280e1b569>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  assert store_complete.isnull().sum()[0] == 11, "You should have found 11 days with missing data, no more no less."


## Exercise 3: Working with timestamps

#### 3.1) Worst day in 2016

What was the worst day in terms of customers in 2016?

In [47]:
# hint: the answer should be a timestamp

# What was the worst day in terms of customers in 2016
mask = store_complete.index.year == 2016
store_2016 = store_complete[mask]
worst_day_2016 = store_2016.customers.idxmin()


In [45]:
expected_hash = '54ca8373016eeb8acb093f60f9d909b8fc1bcc8e37d9f762530df4053bb83a1d'
assert hashlib.sha256(str(worst_day_2016).encode()).hexdigest() == expected_hash

print(f"The worst day in 2016 was {worst_day_2016.day} of {worst_day_2016.month_name()}. Talk about new year's blues !")

The worst day in 2016 was 4 of January. Talk about new year's blues !


#### 3.2) Best Friday

Last Friday there were 3000 customers, and your boss said he's never seen such a high count of customers on a Friday. To check if your boss is correct, can you find the maximum number of customers that we've ever had on a Friday?

- _hint #1: you can use the methods at the bottom of this [page](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) directly on the index_  
- _hint #2: when operating directly on the index, you do not need to use `.dt` to use the methods_

In [49]:

friday_mask = store_complete.index.day_name() == 'Friday'
store_friday = store_complete[friday_mask]
max_customers_Friday = store_friday.customers.max()

In [50]:
expected_hash = 'b134ce47a896876fe3111bfed26cbe06363ede8a60ada5f70fe285d04fc4b7e9'
assert hashlib.sha256(str(int(max_customers_Friday)).encode()).hexdigest() == expected_hash

print(f"Yep! The highest count we ever had on a Friday was {int(max_customers_Friday)} customers. Don't tell your boss.")

Yep! The highest count we ever had on a Friday was 3023 customers. Don't tell your boss.


## Exercise 4: Time series methods

#### 4.1) Shopping rush

A new pandemic has started, and everyone came to buy soap and isopropyl alcohol. Your boss swears to have never seen such an absolute increase in customers from one day to the next - "Yesterday there were 100 customers, today there were 5000."

To confirm if what your boss is saying is true, can you find the maximum increase in customers from one day to the next?

In [53]:
max_increase = store_complete.customers.diff().max()

In [54]:
expected_hash = 'aa4b0d224e2b4488c6e3c5692347a0e26322d86dcb6bf01ab937e15d76037ee4'
assert hashlib.sha256(str(int(max_increase)).encode()).hexdigest() == expected_hash

#### 4.2) Bad month

Despite the shopping rush of the last few days, we had a bad month , with a monthly sum of customers < 45000 . What was the last month we had less than 45000 customers  (`last_bad_month`)?

In [63]:
sum_monthly_customers = store_complete.resample('M').sum()
last_bad_month = sum_monthly_customers[sum_monthly_customers.customers < 45000].index[-1]

<ipython-input-63-2a51124c4005>:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  sum_monthly_customers = store_complete.resample('M').sum()


In [64]:
expected_hash = '50aa98c99f36b286c0269526800c2fc49253f75fcdef0cf02f4db4aee064ddbc'
assert hashlib.sha256(str(last_bad_month).encode()).hexdigest() == expected_hash

---